# Bloc 3 - Analyse prédictive de données structurées par l'intelligence artificielle - Walmart sales

## Introduction

Walmart Inc. is an American multinational retail corporation founded in 1962. As of 2022, Walmart had over 10.000 stores and clubs in 24 countries, operating a chain of hypermarkets, discount department stores, and grocery stores. In 2022, Walmart was rated the world's largest company by revenue, and the world's largest private employer.

### Problematic

Walmart aims at planning and improving its future marketing campaigns.

The company would like to better understand how the sales are influenced by economic indicators.

### Scope

To predict the amount of sales from economic indicators, Walmart provided a dataset containing information about weekly sales for different Walmart stores, as well as other variables such as the unemployment rate or the fuel price.

### Aim and objectives

Overall aim: Build a machine learning model able to estimate the weekly sales in Walmart stores.

Objectives:
- 1 - Make an EDA and preprocess data for machine learning.
- 2 - Train a linear regression model (baseline) and assess its performance.
- 3 - Identify features that are important for the prediction.
- 4 - Avoid overfitting by training a regularized regression model.
- 5 - Fine-tune the regularized regression model.

##
## Methods

### 1 - Library import

### 2 - File reading and basic exploration

The dataset was composed of 150 records of weekly sales for 20 stores. It contained 8 features: the store identification number, the date of the record, the weekly sales, a flag for holidays, the temperature, the fuel price, the consumer price index, and the unemployment rate. 

The initial inspection of the dataset revealed that all features but the store id contained missing values, that the "Date" column could be split to obtain additional features, and that some variables might contain outliers.

### 3 - Exploratory data analysis

An univariate analysis, a bivariate analysis and a correlation matrix were plotted to obtain information on the distribution of each variable, possible correlations and interdependencies between features.

### 4 - Prepocessing

According to what was observed, a first preprocessing was performed with the following steps:
- 1 - Rows containing missing values in the target variable "Weekly_Sales" or in the variables "Date" and "Holiday_Flag" were dropped.
- 2 - Columns recording the year, the month, the day, and the day of the week were created from the "Date" column, which was subsequently dropped.
- 3 - Rows containing outliers (defined as values diverging from the mean by more than three standard deviations) in the features "Temperature", "Fuel_Price", "CPI", and "Unemployment" were dropped.

Data was then processed for machine learning:
- 1 - The target variable was separated from the features.
- 2 - The dataset was divided into train and test sets.
- 3 - For numeric features, missing values were imputed and scaled. Categorical features were encoded.

### 5 - Baseline model (linear regression)

A linear regression model was trained on the train set and its performances were assessed.

### 6 - Baseline model feature importance

The feature importance of the baseline model was displayed from the model coefficients.

### 7 - Regularized regression model (Ridge)

A regularized linear regression model (Ridge) was trained on the train set and its performances were assessed.

### 8 - Regularized model feature importance

The feature importance of the regularized model was displayed from the model coefficients.

### 9 - Regularized model fine-tuning

The regularization strength of the regularized model was fine-tuned. The fine-tuned model was trained on the train set and its performances were assessed.

### 10 - Regularized model feature importance after tuning

The feature importance of the fine-tuned regularized model was displayed from the model coefficients.

### 11 - Results summary

The performance indicators (cross-validated R2, R2 standard deviation, R2 train, R2 test) obtained from the three models were displayed for comparison.

### 12 - Store selection and economic indicators

To investigate whether the selection of stores made by Walmart was the best for the prediction of weekly sales, data from the 6 top stores (having coefficients above 1M for the third model) was extracted and a correlation matrix of the economics indicators was drawned to gain insights on possible correlations with the weekly sales.

##
## Conclusion

The dataset provided by Walmart was small (109 rows after cleaning) but allowed for the production of predictive models and for the identification of the best features for predictions.

The first model (linear regression) already gave good performances, with a R2 of 0.98 on the train set and a R2 of 0.93 on the test set. Unexpectedly, no major overfitting was observed for this model. The second model (Ridge with default lambda equal to 1.0) performed less well, but was improved by the fine-tuning of its regularization strength to a value of 0.01. The resulting R2s on the train and test sets were comparable to the ones obtained with the linear regression model.

Regarding the feature importance, it seemed that for all tested models, stores themselves were the most important factors for the prediction, while economic indicators did not influence much the predictions of weekly sales.

For their future marketing campaigns, it would be advisable for the marketing service of Walmart to focus on the selection of the stores they use for their study if they aim at using economic indicators. As an example, the selection of the six most predictive stores revealed a strong negative correlation between the CPI and the weekly sales (coefficient = -0.72) which could be used in the future.

##
## Code

### 1 - Library import

In [ ]:
### 1 - library import ### ----

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


###
### 2 - File reading and basic exploration

In [ ]:
### 2 - file reading and basic exploration - import dataset ### ----

# read data
data = pd.read_csv("cnm_bloc3-1_data.csv")


In [ ]:
### 2 - file reading and basic exploration - get basic stats ### ----

# print shape of data
print("Number of rows: {}".format(data.shape[0]))
print("Number of columns: {}".format(data.shape[1]))
print()

# display dataset
pd.set_option('display.max_columns', None)
print("Dataset display: ")
display(data.head())
print()

# display basic statistics
print("Basics statistics: ")
data_desc = data.describe(include='all')
display(data_desc)
print()


In [ ]:
### 2 - file reading and basic exploration - get percentage of missing values ### ----

# check wether some columns are full of NaNs
column_nan_full = data.columns[data.isnull().all()]
column_nb = len(column_nan_full)

# get percentage of missing values in columns
percent_nan_col = data.isnull().sum() / data.shape[0] * 100

# check wether some rows are full of NaNs
row_nan_count = pd.Series([data.loc[i,:].isnull().sum() for i in range(0, data.shape[0])])
row_nan_full = row_nan_count.index[row_nan_count == data.shape[1]]
row_nb = len(row_nan_full)

# print report
print("COLUMNS")
print("{} columns out of {} are fully filled with missing values".format(column_nb,data.shape[1]))
print("Percentage of missing values per column:\n{}".format(percent_nan_col))
print()
print("ROWS")
print("{} rows out of {} are fully filled with missing values".format(row_nb,data.shape[0]))


###
### 3 - Exploratory data analysis

In [ ]:
### 3 - exploratory data analysis - plot univariate analysis ### ----

# set figure to make subplots
fig1 = make_subplots(
    rows = 2,
    cols = 4,
    subplot_titles = (
        "A. Weekly sales",
        "B. Temperature",
        "C. Fuel price",
        "D. CPI",
        "E. Unemployment rate",
        "F. Holidays",
        "G. Stores"),
    column_widths = [0.20, 0.20, 0.20, 0.20],
    horizontal_spacing = 0.15)

# plot distribution of each numeric variable
features_num = ["Weekly_Sales", "Temperature", "Fuel_Price", "CPI", "Unemployment"]
[fig1.add_trace(go.Histogram(
    x = data[features_num[i]],
    marker_color = px.colors.qualitative.Vivid[i]),
    row = 1, col = i+1) for i in [0, 1, 2, 3]]
[fig1.add_trace(go.Histogram(
    x = data[features_num[i]],
    marker_color = px.colors.qualitative.Vivid[i]),
    row = 2, col = i-3) for i in [4]]

# plot categorical variables
holidays = data["Holiday_Flag"].value_counts()
fig1.add_trace(go.Bar(
    x = ["No", "Yes"],
    y = holidays.values,
    marker_color = px.colors.qualitative.Vivid[6:]),
    row = 2, col = 2)
stores = data["Store"].value_counts()
fig1.add_trace(go.Bar(
    x = stores.index,
    y = stores.values,
    marker_color = px.colors.qualitative.Vivid[8]),
    row = 2, col = 3)

# update layout
fig1.update_annotations(font_size = 15)
fig1.update_xaxes(tickfont = dict(size = 10))
fig1.update_yaxes(tickfont = dict(size = 10))
fig1.update_layout(
        margin = dict(l = 90, t= 120),
        title_text = "Figure 1. Univariate analysis",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        xaxis = dict(title = go.layout.xaxis.Title(text = "Weekly sales (Dollars)", font_size = 10)),
        xaxis2 = dict(title = go.layout.xaxis.Title(text = "Temperature (Degrees Fahrenheit)", font_size = 10)),
        xaxis3 = dict(title = go.layout.xaxis.Title(text = "Fuel price (Dollars)", font_size = 10)),
        xaxis4 = dict(title = go.layout.xaxis.Title(text = "CPI (AU)", font_size = 10)),
        xaxis5 = dict(title = go.layout.xaxis.Title(text = "Unemployment rate (Percent)", font_size = 10)),
        xaxis6 = dict(title = go.layout.xaxis.Title(text = "Holiday period", font_size = 10)),
        xaxis7 = dict(title = go.layout.xaxis.Title(text = "Store id", font_size = 10)),
        yaxis = dict(range = [0, 24], tickvals = [0, 5, 10, 15, 20]),
        yaxis2 = dict(range = [0, 24], tickvals = [0, 5, 10, 15, 20]),
        yaxis3 = dict(range = [0, 36], tickvals = [0, 10, 20, 30]),
        yaxis4 = dict(range = [0, 72], tickvals = [0, 20, 40, 60]),
        yaxis5 = dict(range = [0, 30], tickvals = [0, 5, 10, 15, 20, 25]),
        yaxis6 = dict(title = go.layout.yaxis.Title(text = "Count", font_size = 10, standoff = 0), 
            range = [0, 180], tickvals = [0, 50, 100, 150]),
        yaxis7 = dict(title = go.layout.yaxis.Title(text = "Count", font_size = 10, standoff = 0), 
            range = [0, 18], tickvals = [0, 5, 10, 15]),
        bargroupgap = 0.4,
        showlegend = False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 600)

fig1.show()


In [ ]:
### 3 - exploratory data analysis - plot bivariate analysis ### ----

# plot pairwise dependencies
fig2 = go.Figure(data = go.Splom(
        dimensions = [dict(label = "Weekly sales", values = data["Weekly_Sales"]),
                dict(label = "Holidays", values = data["Holiday_Flag"]),
                dict(label = "Temperature", values = data["Temperature"]),
                dict(label = "Fuel price", values = data["Fuel_Price"]),
                dict(label = "CPI", values = data["CPI"]),
                dict(label = "Unemployment", values = data["Unemployment"]),
                dict(label = "Stores", values = data["Store"])],
        marker = dict(color = px.colors.qualitative.Vivid[1], size = 5),
        diagonal = dict(visible = False)))

# update layout
fig2.update_layout(
        margin = dict(l = 105, t = 100),
        title_text = "Figure 2. Bivariate analysis",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,   
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 800)

fig2.show()


In [ ]:
### 3 - exploratory data analysis - plot correlation matrix ### ----

# get correlation matrix
corr_matrix = data.loc[:,features_num].corr().round(2)

# plot correlation matrix
fig3 = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())

# update layout
fig3.update_layout(
        margin = dict(l = 150, b = 40),
        title_text = "Figure 3. Correlation matrix",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,   
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig3.show()


###
### 4 - Preprocessing

In [ ]:
### 4 - preprocessing - clean data ### ----

# copy data for safety
data1 = data.copy()


# part 1 - drop rows where values are missing (for date, weekly sales and holiday flag)

# get index of data to drop
mask_drop = (data1["Date"].isnull()) | (data1["Weekly_Sales"].isnull()) | (data1["Holiday_Flag"].isnull())
index_drop = data1.index[mask_drop]

# drop data
data1 = data1.drop(index_drop, axis = 0)

# print report
print("Number of rows with missing values that were dropped: {}"
    .format(len(index_drop)))
print()

# part 2 - create usable features from the Date column and drop date

# format date column
data1["Date"] = pd.to_datetime(data1["Date"], infer_datetime_format = True)

# create features year, month, day, day of week
data1["year"] = data1["Date"].dt.year
data1["month"] = data1["Date"].dt.month
data1["day"] = data1["Date"].dt.day
data1["day_of_week"] = data1["Date"].dt.day_of_week
data1 = data1.drop(["Date"], axis = 1)

# display new dataset
print("Dataset display: ")
display(data1.head())
print("Data shape: {}".format(data1.shape))
print()


# part 3 - drop rows containing outliers

# set columns to check
columns_tocheck = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]

# initialise variables to store number of rows dropped
drop_nb = 0

# loop through columns
for i in columns_tocheck:

    # set bounds to identify outliers
    column_mean = data1[i].mean()
    column_std = data1[i].std()
    lower_bond = column_mean - 3 * column_std
    upper_bond = column_mean + 3 * column_std

    # get index of rows to drop
    mask_drop = (data1[i] < lower_bond) | (data1[i] > upper_bond)
    index_drop = data1.index[mask_drop]

    # drop rows
    data1 = data1.drop(index_drop, axis = 0)

    # count rows that were droped
    drop_nb += len(index_drop)

# print report
print("Number of rows with outliers that were dropped: {}".format(drop_nb))


In [ ]:
### 4 - preprocessing - process data for  machine learning ### ----

# separate target variable Y from features X
X = data1.drop(["Weekly_Sales"], axis = 1)
Y = data1["Weekly_Sales"]

# divide dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# create preprocessor object from pipelines for numeric and categorical features
features_num = ["Temperature", "Fuel_Price", "CPI", "Unemployment", "year", "month", "day", "day_of_week"]
features_cat = ["Store", "Holiday_Flag"]
numeric_transformer = Pipeline(steps = [
    ("imputer", KNNImputer()),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline(steps = [
    ("encoder", OneHotEncoder(drop = "first"))
])
preprocessor = ColumnTransformer(transformers = [
    ("num", numeric_transformer, features_num),
    ("cat", categorical_transformer, features_cat)
])

# impute missing values and scale numeric features, encode categorical features
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


###
### 5 - Baseline model (linear regression)

In [ ]:
### 5 - baseline model (linear regression) - train model and asess performance ### ----

# train model
regressor1 = LinearRegression()
regressor1.fit(X_train, Y_train)

# make predictions on train and test sets
Y_train_pred1 = regressor1.predict(X_train)
Y_test_pred1 = regressor1.predict(X_test)

# perform 5-fold cross-validation to evaluate the generalized R2 score 
scores1 = cross_val_score(regressor1, X_train, Y_train, cv = 5)
print('Cross-validated R2 score: ', scores1.mean())
print('Standard deviation: ', scores1.std())
print()

# assess performance and print report
r2_train1 = r2_score(Y_train, Y_train_pred1)
r2_test1 = r2_score(Y_test, Y_test_pred1)
print("R2 score on training set: ", r2_train1)
print("R2 score on test set: ", r2_test1)


###
### 6 - Baseline model feature importance

In [ ]:
### 6 - baseline model feature importance ### ----

# get column names from the preprocessor
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: 
    if name == 'num': 
        features = features_list 
    else: 
        features = pipeline.named_steps['encoder'].get_feature_names_out() 
    column_names.extend(features)

# store coefficients in a dataframe
coefs1 = pd.DataFrame(index = range(0,len(regressor1.coef_)), columns = ["features", "coefficients"])
coefs1["features"] = column_names
coefs1["coefficients"] = abs(regressor1.coef_)

# get feature importance
feature_importance1 = coefs1.sort_values("coefficients", ascending = False).reset_index(drop = True)

# plot feature importance
fig4 = go.Figure([go.Bar(
    x = feature_importance1.loc[:,"features"],
    y = feature_importance1.loc[:,"coefficients"],
    marker_color = px.colors.qualitative.Vivid)])

# update layout
fig4.update_xaxes(tickfont = dict(size = 10), tickangle = 90)
fig4.update_yaxes(tickfont = dict(size = 10))
fig4.update_layout(
        margin = dict(l = 120),
        title_text = "Figure 4. Baseline model feature importance",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        xaxis = dict(title = "Features"),
        yaxis = dict(title = "Coefficients", range = [-50000, 1600000], tickvals = [0, 500000,1000000,1500000]),
        showlegend = False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig4.show()


###
### 7 - Regularized regression model (Ridge)

In [ ]:
### 7 - regularized regression model (Ridge) ### ----

# train model
regressor2 = Ridge()
regressor2.fit(X_train, Y_train)

# make predictions on train and test sets
Y_train_pred2 = regressor2.predict(X_train)
Y_test_pred2 = regressor2.predict(X_test)

# perform 5-fold cross-validation to evaluate the generalized R2 score 
scores2 = cross_val_score(regressor2, X_train, Y_train, cv = 5)
print('Cross-validated R2 score: ', scores2.mean())
print('Standard deviation: ', scores2.std())
print()

# assess performance and print report
r2_train2 = r2_score(Y_train, Y_train_pred2)
r2_test2 = r2_score(Y_test, Y_test_pred2)
print("R2 score on training set: ", r2_train2)
print("R2 score on test set: ", r2_test2)


###
### 8 - Regularized model feature importance

In [ ]:
### 8 - regularized model feature importance ### ----

# store coefficients in a dataframe
coefs2 = pd.DataFrame(index = range(0,len(regressor2.coef_)), columns = ["features", "coefficients"])
coefs2["features"] = column_names
coefs2["coefficients"] = abs(regressor2.coef_)

# get feature importance
feature_importance2 = coefs2.sort_values("coefficients", ascending = False).reset_index(drop = True)

# plot feature importance
fig5 = go.Figure([go.Bar(
    x = feature_importance2.loc[:,"features"],
    y = feature_importance2.loc[:,"coefficients"],
    marker_color = px.colors.qualitative.Vivid)])

# update layout
fig5.update_xaxes(tickfont = dict(size = 10), tickangle = 90)
fig5.update_yaxes(tickfont = dict(size = 10))
fig5.update_layout(
        margin = dict(l = 120),
        title_text = "Figure 5. Regularized model feature importance",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        xaxis = dict(title = "Features"),
        yaxis = dict(title = "Coefficients", range = [-50000, 1600000], tickvals = [0, 500000,1000000,1500000]),
        showlegend = False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig5.show()


###
### 9 - Regularized model fine-tuning

In [ ]:
### 9 - regularized model fine-tuning ### ----

# tune lambda with gridsearch
params = {
    'alpha': [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0]
}
gridsearch = GridSearchCV(regressor2, param_grid = params, cv = 5)
gridsearch.fit(X_train, Y_train)

# print report
print("Best hyperparameter: ", gridsearch.best_params_)
print()

# train model
regressor3 = Ridge(alpha = gridsearch.best_params_["alpha"])
regressor3.fit(X_train, Y_train)

# perform 5-fold cross-validation to evaluate the generalized R2 score 
scores3 = cross_val_score(regressor3, X_train, Y_train, cv = 5)
print('Cross-validated R2 score: ', scores3.mean())
print('Standard deviation: ', scores3.std())
print()

# make predictions on train and test sets
Y_train_pred3 = regressor3.predict(X_train)
Y_test_pred3 = regressor3.predict(X_test)

# assess performance and print report
r2_train3 = r2_score(Y_train, Y_train_pred3)
r2_test3 = r2_score(Y_test, Y_test_pred3)
print("R2 score on training set: ", r2_train3)
print("R2 score on test set: ", r2_test3)


###
### 10 - Regularized model feature importance after tuning

In [ ]:
### 10 - regularized model feature importance after tuning ### ----

# store coefficients in a dataframe
coefs3 = pd.DataFrame(index = range(0,len(regressor3.coef_)), columns = ["features", "coefficients"])
coefs3["features"] = column_names
coefs3["coefficients"] = abs(regressor3.coef_)

# get feature importance
feature_importance3 = coefs3.sort_values("coefficients", ascending = False).reset_index(drop = True)

# plot feature importance
fig6 = go.Figure([go.Bar(
    x = feature_importance3.loc[:,"features"],
    y = feature_importance3.loc[:,"coefficients"],
    marker_color = px.colors.qualitative.Vivid)])

# update layout
fig6.update_xaxes(tickfont = dict(size = 10), tickangle = 90)
fig6.update_yaxes(tickfont = dict(size = 10))
fig6.update_layout(
        margin = dict(l = 120),
        title_text = "Figure 6. Regularized model feature importance after tuning",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        xaxis = dict(title = "Features"),
        yaxis = dict(title = "Coefficients", range = [-50000, 1600000], tickvals = [0, 500000,1000000,1500000]),
        showlegend = False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig6.show()


###
### 11 - Results summary

In [ ]:
### 11 - Results summary ### ----

# store results in a dataframe
results = pd.DataFrame(index = ["Linear regression", "Ridge lambda=1", "Ridge lambda=0.01"], 
    columns = ["Cross-validated R2", "R2 standard deviation", "R2 train", "R2 test"])
results["Cross-validated R2"] = [scores1.mean(), scores2.mean(), scores3.mean()]
results["R2 standard deviation"] = [scores1.std(), scores2.std(), scores3.std()]
results["R2 train"] = [r2_train1, r2_train2, r2_train3]
results["R2 test"] = [r2_test1, r2_test2, r2_test3]

# print results summary
print("Results summary: ")
display(results)


###
### 12 - Store selection and economic indicators

In [ ]:
### 12 - store selection and economic indicators ### ----

# copy data for safety
data2 = data1.copy()

# get top 6 stores with the highest coefficients from model3 (coefficients above 1M)
stores_desc = [5.0, 3.0, 9.0, 16.0, 7.0, 15.0]

# extract data for these stores
features_totest = ["Weekly_Sales", "Temperature", "Fuel_Price", "CPI", "Unemployment"]
data2 = data2.loc[data2["Store"].isin(stores_desc),features_totest]

# get correlation matrix
corr_matrix2 = data2.corr().round(2)

# plot correlation matrix
fig7 = ff.create_annotated_heatmap(corr_matrix2.values,
                                  x = corr_matrix2.columns.tolist(),
                                  y = corr_matrix2.index.tolist())

# update layout
fig7.update_layout(
        margin = dict(l = 150, b = 40),
        title_text = "Figure 7. Correlation matrix",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,   
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig7.show()
